In [1]:
import scanpy as sc
from benchutils import load_adata

import sys
sys.path.append("../src/")

from refcm import RefCM


In [2]:
%load_ext autoreload
%autoreload 2

Loading the relevant datasets into memory

In [3]:
ds = ["MTG", "ALM", "VISp"]

# load_adata applies basic preprocessing -- lognorm in .X, counts in .layers['counts'] and such
ds = {s: load_adata(f"../data/{s}.h5ad") for s in ds}

In [5]:
ds['MTG']

AnnData object with n_obs × n_vars = 14055 × 16024
    obs: 'labels3', 'labels34'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    layers: 'counts'

Sample run annotating MTG given ALM as reference

In [6]:
q, ref = "MTG", "VISp"

# create RefCM object
rcm = RefCM()

# set reference
rcm.setref(ds[ref], "labels34")

# annotate query dataset
rcm.annotate(ds[q], "labels34")

ds[q].obs

|████████████████| [100.00%] : 00:05


,labels3,labels34,refcm
F1S4_160106_001_B01,Inhibitory,Sst 1,Sst 1
F1S4_160106_001_C01,Excitatory,Exc L5/6 IT 3,Exc L5/6 IT 3
F1S4_160106_001_E01,Excitatory,Exc L6 CT,Exc L6 CT
F1S4_160106_001_G01,Excitatory,Exc L6b,Exc L6b
F1S4_160106_001_H01,Excitatory,Exc L4/5 IT,Exc L4/5 IT
...,...,...,...
F2S4_170405_060_A01,Excitatory,Exc L6 IT 1,Exc L6 IT 1
F2S4_170405_060_B01,Excitatory,Exc L6 CT,Exc L6 CT
F2S4_170405_060_C01,Excitatory,Exc L6 IT 2,Exc L6 IT 2
F2S4_170405_060_E01,Excitatory,Exc L5/6 NP,Exc L5/6 NP


In [13]:
(ds[q].obs['refcm'] == ds[q].obs['labels34']).mean()

np.float64(1.0)

Annotating all Allen-Brain datasets against each other

In [15]:
rcm = RefCM()

for i, q_id in enumerate(ds):
    for ref_id in [d for d in ds if d != q_id]:
        rcm.setref(ds[ref_id], "labels34")
        rcm.annotate(ds[q_id], "labels34")
        acc = (ds[q_id].obs['refcm'] == ds[q_id].obs['labels34']).mean()
        print(f"{q_id:>6} | {ref_id:<6} : {acc:.2f}")

|████████████████| [100.00%] : 00:03


   MTG | ALM    : 1.00


|████████████████| [100.00%] : 00:05


   MTG | VISp   : 1.00


|████████████████| [100.00%] : 00:04


...

   ALM | VISp   : 1.00


|████████████████| [100.00%] : 00:06


  VISp | MTG    : 1.00


|████████████████| [100.00%] : 00:02


  VISp | ALM    : 1.00
